# **Tugas Pengantar AI 3**

### **Kelompok 14 (IF-43-06) :**
- Ivan Ryansyach (1301194150)
- Muhammad Tsaqif Ammar (1301194222)
- Mar'i Fauzan Rambe (1301194204)

### **Deskripsi Tugas**

Diberikan file `mobil.xls` berupa himpunan data 17 data mobil dengan 5 (lima) atribut:

1. **Ukuran** akan  bernilai  dari  0-10  dimana  semakin  besar  angkanya  semakin  besar  ukuran mobil.
2. **Kenyamanan** akan  bernilai  0-10  dimana  semakin  besar  angkanya  semakin  nyaman mobilnya.
3. **Irit** akan bernilai 0-10 dimana  semakin besar angkanya akan semakin irit.
4. **Kecepatan** akan bernilai 0-10 dimana semakin besar akan semakin cepat.
5. **Harga** akan bernilai ratus juta, jika bernilai 1 maka akan senilai 100 juta, 3,75 artinya mobil akan berharga 375 juta.

Bangunlah sebuah sistem rekomendasi mobil berbasis kNN untuk memilih 3 (tiga) mobil terbaik sesuai dengan inputan user. Sistem membaca masukan file `mobil.xls` dan mengeluarkan output berupa sebuah file `rekomendasi.xls` yang berisi satu vektor kolom berisi 3 (tiga) baris string yang menyatakan nama mobil yang direkomendasikan pada file `mobil.xls`.

### **Library yang dibutuhkan**


In [ ]:
import pandas as pd
from math import sqrt
from IPython.display import display

### **Mengimport Data Mobil**

In [ ]:
# Import data mobil
data_mobil = pd.read_excel("https://github.com/tsaqifammar/knn-car-recommendation-system/blob/main/mobil.xls?raw=true")
atribut = list(data_mobil.columns[1:])

print(atribut)
display(data_mobil)

['Ukuran', 'Kenyamanan', 'Irit', 'Kecepatan', 'Harga (Ratus Juta)']


,Nama Mobil,Ukuran,Kenyamanan,Irit,Kecepatan,Harga (Ratus Juta)
0,Toyota Agya,4,4,9,6,1.00
1,Daihatsu Alya,4,3,9,6,1.10
2,Toyota Avanza,6,5,6,6,2.00
3,Daihatsu Xenia,6,4,6,6,1.75
4,Xpander,7,7,6,7,2.25
5,Livina,7,7,6,7,2.10
6,Karimun,3,4,10,5,1.20
7,Toyota Innova,8,8,5,7,4.00
8,Alphard,9,10,4,8,10.00
9,Toyota Vios,5,7,9,8,2.50


### **Prapemrosesan Data**
Disini dilakukan normalisasi data dengan teknik [min-max scaling](https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)). Berikut rumus normalisasi nya :
$$x' = \frac{x - min(x)}{max(x) - min(x)}$$

In [ ]:
min_atr = dict(data_mobil[atribut].min())
max_atr = dict(data_mobil[atribut].max())
def normalize(row):
  for i in atribut:
    row[i] = (row[i] - min_atr[i])/(max_atr[i] - min_atr[i])
  return row

data_mobil = data_mobil.apply(lambda row : normalize(row), axis=1)
display(data_mobil)

,Nama Mobil,Ukuran,Kenyamanan,Irit,Kecepatan,Harga (Ratus Juta)
0,Toyota Agya,0.142857,0.142857,0.833333,0.25,0.021739
1,Daihatsu Alya,0.142857,0.000000,0.833333,0.25,0.032609
2,Toyota Avanza,0.428571,0.285714,0.333333,0.25,0.130435
3,Daihatsu Xenia,0.428571,0.142857,0.333333,0.25,0.103261
4,Xpander,0.571429,0.571429,0.333333,0.50,0.157609
5,Livina,0.571429,0.571429,0.333333,0.50,0.141304
6,Karimun,0.000000,0.142857,1.000000,0.00,0.043478
7,Toyota Innova,0.714286,0.714286,0.166667,0.50,0.347826
8,Alphard,0.857143,1.000000,0.000000,0.75,1.000000
9,Toyota Vios,0.285714,0.571429,0.833333,0.75,0.184783


### **Fungsi Perhitungan Jarak**

Dalam perhitungan jarak, akan dibandingkan hasilnya dari 4 metode perhitungan berikut : 
- Euclidean Distance
- Manhattan Distance
- Minkowski Distance
- Supremum Distance

In [ ]:
# p1 dan p2 adalah list of integers
# p1 = user input, p2 = data mobil
def euclidean(p1, p2):
  dist = 0
  for i in atribut:
    dist += (p1[i]-p2[i])**2
  return sqrt(dist)

def manhattan(p1, p2):
  dist = 0
  for i in atribut:
    dist += abs(p1[i]-p2[i])
  return dist

def minkowski(p1, p2):
  h = 4
  dist = 0
  for i in atribut:
    dist += abs(p1[i]-p2[i])**h
  return dist**(1/h)

def supremum(p1, p2):
  return max([abs(p1[i]-p2[i]) for i in atribut])

fungsi_jarak = [euclidean, manhattan, minkowski, supremum]

### **k-Nearest Neighbour**

Mencari $k$ tetangga terdekat dengan cara menghitung jarak antara inputan user dengan tiap baris `data_mobil`. Lalu mengurutkan data berdasarkan jarak tersebut. Setelah itu, ambil $k$ data pertamanya.

In [ ]:
def knn(input, k, calc_distance):
  nama_fungsi = calc_distance.__name__

  # Normalisasi input
  input = normalize(input)

  # Membuat kolom Jarak, yaitu jarak antara inputan user dengan tiap baris data_mobil
  data_mobil[nama_fungsi] = data_mobil.apply(lambda row : calc_distance(input, row), axis=1)

  # Return 3 mobil dengan jarak terdekat
  return data_mobil.sort_values(by=[nama_fungsi], ascending=True)[0:3]

### **Program Utama**

In [ ]:
# Menginput data user
data_user = dict()
print("Masukkan nilai kriteria mobil yang anda inginkan (1-10):\n")
for lab in atribut:
  print(f"{lab}: ", end="")
  data_user[lab] = float(input())

# Mendapatkan 3 mobil terbaik untuk setiap metode penghitungan jarak
top_3s = []
k = 3
for i in range(len(fungsi_jarak)):
  top_3s.append(knn(data_user, k, fungsi_jarak[i]))

# Print hasil untuk setiap metode penghitungan jarak
for i in range(len(fungsi_jarak)):
  nama_fungsi = fungsi_jarak[i].__name__
  show_col = ["Nama Mobil", nama_fungsi]
  print(f'\n======== {nama_fungsi.title()} =======')
  display(top_3s[i][show_col])

Masukkan nilai kriteria mobil yang anda inginkan (1-10):

Ukuran: 4
Kenyamanan: 7
Irit: 8
Kecepatan: 5
Harga (Ratus Juta): 1.8

======== Euclidean =======


,Nama Mobil,euclidean
0,Toyota Agya,0.530578
6,Karimun,0.565196
2,Toyota Avanza,0.580818



======== Manhattan =======


,Nama Mobil,manhattan
6,Karimun,3.822135
3,Daihatsu Xenia,3.893822
1,Daihatsu Alya,3.894599



======== Minkowski =======


,Nama Mobil,minkowski
3,Daihatsu Xenia,1.901376
2,Toyota Avanza,1.908348
16,Suzuki Carry,1.923901



======== Supremum =======


,Nama Mobil,supremum
16,Suzuki Carry,1.640625
6,Karimun,1.793210
11,Toyota Hiace,1.890625


### **Export 3 Mobil Rekomendasi**

Disini kami akan mengexport 3 mobil rekomendasi dengan metode penghitungan jarak Euclidean Distance.

*Note: Jalankan dulu program utama nya*

In [ ]:
top3_euc = top_3s[0]
top3_euc.drop(atribut + ["euclidean", "manhattan", "minkowski", "supremum"], axis=1)
top3_euc.to_excel("rekomendasi.xls", index=False, header=False)